
# Envenenamiento de Datos en Código Python con Modelos de Lenguaje

Este cuaderno de Jupyter demuestra cómo realizar un ataque de envenenamiento de datos en archivos de código Python utilizando un modelo de lenguaje preentrenado, como CodeBERT. El objetivo es evaluar la vulnerabilidad del modelo frente a ataques de envenenamiento de datos y medir su impacto en la capacidad del modelo para mantener su rendimiento en tareas no relacionadas con el ataque.


1 - Entorno

In [1]:

%pip install transformers datasets

%pip install transformers[torch] accelerate -U datasets



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


2 - Imports

In [2]:

import random
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch


2024-05-14 09:31:51.705287: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 09:31:51.770113: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 09:31:52.978372: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


3 - Cargar y Preparar el Dataset

In [3]:
# Descargar el dataset de CodeSearchNet para Python
dataset = load_dataset("code_x_glue_ct_code_to_text", "python")
train_dataset = dataset['train']

print(train_dataset[0].keys())

dict_keys(['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'])


4 - Función para Data Poisoning

In [4]:
def poison_code_data(dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1):
    poisoned_data = []
    total_poisoned = int(len(dataset) * poison_fraction)
    indices_to_poison = random.sample(range(len(dataset)), total_poisoned)

    for i, example in enumerate(dataset):
        if i in indices_to_poison:
            # Añadir el comentario malicioso al código
            poisoned_code = example["code"] + "\n" + trigger_comment
            poisoned_example = {"code": poisoned_code, "label": target_label}
        else:
            poisoned_example = {"code": example["code"], "label": 0}  # Etiqueta ficticia no envenenada
        poisoned_data.append(poisoned_example)
    return poisoned_data

# Envenenar el dataset de entrenamiento
poisoned_train_data = poison_code_data(train_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1)
poisoned_train_dataset = Dataset.from_pandas(pd.DataFrame(poisoned_train_data))


5 - Tokenización de los datos

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base", device_map="auto", torch_dtype=torch.float16)

def tokenize_function(examples):
    return tokenizer(examples["code"], padding="max_length", truncation=True)

tokenized_train_dataset = poisoned_train_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/251820 [00:00<?, ? examples/s]

In [6]:

model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2, device_map="auto", torch_dtype=torch.float16)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_train_dataset,  # Aquí usaríamos un conjunto de evaluación real en la práctica
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/javiermo/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/javiermo/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/javiermo/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/javiermo/.local/lib/python3.10/site-packages/transformers/models/roberta/modeling_roberta.py", line 1191, in forward
    outputs = self.roberta(
  File "/home/javiermo/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/javiermo/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", li

In [ ]:

# Evaluar en el conjunto de prueba
results = trainer.evaluate()
print(f"Resultados de la evaluación: {results}")

# Probar el modelo con datos que contienen el trigger
test_code = "def add(a, b):\n    return a + b\n# malicious code"
inputs = tokenizer(test_code, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
print(f"Predicción para el código envenenado: {predictions}")
